# Ujednoznacznianie sensu słów (WSD) — Rozwiązania zadań

Wykorzystując **Słowosieć 3.0**, wykonaj poniższe zadania.

Link: https://clarin-pl.eu/dspace/bitstream/handle/11321/273/plwordnet_3_0.7z?sequence=2&isAllowed=y

## Zadanie 1

Dla każdej części mowy (*part-of-speech*) wskaż lemat, który posiada najwięcej znaczeń (jednostek leksykalnych).

**Wskazówka**

Część mowy jest atrybutem jednostki leksykalnej i jest zapisania w atrybucie pos.

```
<lexical-unit id="1599" name="ekran" pos="rzeczownik" tagcount="718"
    domain="msc"
    desc="##K: og. ##D: płaszczyzna, na którą rzuca się obraz, np. w kinie."
    workstate="Sprawdzone" source="systemowy" variant="3"/>
```

W Słowosieci występują następujące części mowy:
```
{'czasownik',
 'czasownik pwn',
 'przymiotnik',
 'przymiotnik pwn',
 'przysłówek',
 'przysłówek pwn',
 'rzeczownik',
 'rzeczownik pwn'}
 ```

### Rozwiązanie

In [1]:
!wget -O plwordnet_3_0.7z https://clarin-pl.eu/dspace/bitstream/handle/11321/273/plwordnet_3_0.7z?sequence=2&isAllowed=y

--2021-01-05 06:05:37--  https://clarin-pl.eu/dspace/bitstream/handle/11321/273/plwordnet_3_0.7z?sequence=2
Resolving clarin-pl.eu (clarin-pl.eu)... 156.17.135.38
Connecting to clarin-pl.eu (clarin-pl.eu)|156.17.135.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36119839 (34M) [application/octet-stream]
Saving to: ‘plwordnet_3_0.7z’

plwordnet_3_0.7z    100%[===================>]  34.45M  15.1MB/s    in 2.3s    

2021-01-05 06:05:40 (15.1 MB/s) - ‘plwordnet_3_0.7z’ saved [36119839/36119839]



In [2]:
!7z x plwordnet_3_0.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 36119839 bytes (35 MiB)

Extracting archive: plwordnet_3_0.7z
--
Path = plwordnet_3_0.7z
Type = 7z
Physical Size = 36119839
Headers Size = 313
Method = LZMA:25
Solid = +
Blocks = 1

  0%      5% 1 - plwordnet_3_0/plwordnet-3.0-visdisc.xml                                                 12% 1 - plwordnet_3_0/plwordnet-3.0-visdisc.xml                                                 17% 1 - plwordnet_3_0/plwordnet-3.0-visdisc.xml                                               

In [3]:
import xml.etree.ElementTree as ET

tree = ET.fromstring(open("plwordnet_3_0/plwordnet-3.0.xml", "r").read())

In [15]:
import re
from collections import namedtuple

LexicalUnit = namedtuple('LexicalUnit', 'id name desc pos')
definition_regex = re.compile("##D:([^#<[{]{3,})")

def parse_description(desc: str) -> str:
  definitions = []
  for m in definition_regex.finditer(desc):
    definitions.append(m.group(1).strip())
  return ", ".join(definitions)

lexical_units = {}

for unit in tree.iter('lexical-unit'):
    id = unit.attrib.get('id')
    name = unit.attrib.get('name')
    desc = unit.attrib.get('desc')
    pos = unit.attrib.get('pos')
    definitions = parse_description(unit.attrib.get('desc'))
    lexical_units[id] = LexicalUnit(id, name, definitions, pos)

len(lexical_units)

477881

In [6]:
lexems_by_pos = {}

for lu in lexical_units.values():
  lexems_by_pos.setdefault(lu.pos, {}).setdefault(lu.name, []).append(lu)

print(lexems_by_pos.keys())

dict_keys(['przymiotnik', 'rzeczownik pwn', 'przymiotnik pwn', 'przysłówek pwn', 'rzeczownik', 'przysłówek', 'czasownik pwn', 'czasownik'])


In [12]:
for pos, lexems in sorted(lexems_by_pos.items()):
  lexems_sorted = sorted(lexems.items(), key=lambda name_items: len(name_items[1]), reverse=True)
  top = lexems_sorted[0]
  print(f"{pos:15}: {len(top[1]):>3}  {top[0]}")

czasownik      :  26  brać
czasownik pwn  :  59  break
przymiotnik    :  29  czysty
przymiotnik pwn:  25  heavy
przysłówek     :  19  czysto
przysłówek pwn :  13  well
rzeczownik     :  24  branie
rzeczownik pwn :  33  head


## Zadanie 2

W części praktycznej została użyta metoda `similarity` z pakietu spaCy do porównania kontekstu słowa z dostępnymi definicjami dla lematu *ekran*. Do reprezentacji zdania były wykorzystane wektory osadzeń, które są uśrednieniem wektorów dla poszczególnych słów. Przy uśrednianiu brane są pod uwagę wszystkie słowa, nawet te, które nie niosą istotnej informacji lub są na tyle częste i występują w wielu definicjach, przez co sztucznie zawyżają podobieństwo. Zaproponuj, w jaki sposób można pominąć słowa nieistotne, aby poprawić wynik ujednoznacznienia dla słow ekran w poniższym zdaniu.

In [27]:
text = "Jak włączyłem komputer to na ekranie wyświetlił się komunikat o błędzie"

Wynik bazowy:

```
 0.62    4879  płaszczyzna, na którą rzuca się obraz, np. w kinie.
 0.61     799  wyświetlacz; powierzchnia, na której jest coś wyświetlane.
 0.40  238951  duża naszywka z nazwą zespołu, która pasuje na środkowy klin jeansowej kurtki lub kamizelki.
 0.35    8264  osłona pełniąca funkcję zabezpieczenia, izolacji przed czymś szkodliwym.
 0.29  248273  rodzaj zasłony (z tkaniny, skóry, żelaza), która bywa ustawiana przed kominkiem.
 ```

**Wskazówka**

Wykorzystaj poznany wcześniej sposób do ważenia terminów w kolekcji dokumentów. Co będzie Twoją kolekcją tekstów w tym zadaniu?

### Rozwiązanie

In [18]:
definitions = [lu.desc for lu in lexical_units.values() if len(lu.desc) > 1]
print(len(definitions))
print(*definitions[:10], sep="\n")

125454
taki, który się powtarza lub cechuje się wielkością zależną od liczby powtórzeń charakterystycznego elementu.
oznaczenie stopnia - jednostki miary kąta płaskiego, równej 1/360 kąta pełnego czyli 1/90 kąta prostego.
symbol stopni Celsjusza.
jedna ze skal pomiaru temperatury, używana do pomiaru w krajach stosujących imperialne jednostki miar do połowy XX wieku, nadal stosowana w USA, Kajmanach, Bahamach oraz Belize; obecnie skalę Fahrenheita definiuje się przez porównanie ze skalą Celsjusza - 32 °F = 0 °C; 212 °F = 100 °C.
uncja; jednostka masy równa 31,1034768 grama.
grupa funkcyjna w chemii organicznej o ogólnym wzorze: −C=N−R, występująca w iminach i ich pochodnych.
symbol dolara.
skrót/symbol dolara liberyjskiego.
skrót/symbol dolara Kiribati.
skrót/symbol dolara Tuvalu.


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()     # token_pattern='(?u)\b\w\w+\b',
X = vectorizer.fit_transform(definitions)

# vectorizer.idf_ — wartości idf
# vectorizer.get_feature_names() — słowa

idf_words = [t for t in zip(vectorizer.idf_, vectorizer.get_feature_names())]
idf_words[:10]

[(11.64109011884875, '00'),
 (7.621708595100104, '000'),
 (12.046555226956913, '00000027'),
 (12.046555226956913, '000001'),
 (10.542477830180639, '001'),
 (12.046555226956913, '005'),
 (10.341807134718488, '01'),
 (12.046555226956913, '01163'),
 (12.046555226956913, '01646'),
 (12.046555226956913, '02214129')]

In [22]:
idf_words = sorted(idf_words)
print(*idf_words[:10], sep="\n")
print("...")
print(*idf_words[-10:], sep="\n")

(2.733252211922906, 'który')
(2.753621556676878, 'taki')
(2.9432990775904053, 'się')
(3.0413343204506895, 'na')
(3.057921938508979, 'do')
(3.189538278685949, 'lub')
(3.1916762498538014, 'jest')
(3.581076471227355, 'związany')
(3.6601543257906997, 'rodziny')
(3.9094518373176124, 'gatunek')
...
(12.046555226956913, 'żłobkowanej')
(12.046555226956913, 'żłobkowaną')
(12.046555226956913, 'żłobu')
(12.046555226956913, 'żłożony')
(12.046555226956913, 'żłób')
(12.046555226956913, 'δi')
(12.046555226956913, 'μb')
(12.046555226956913, 'μg')
(12.046555226956913, 'μn')
(12.046555226956913, 'μrad')


In [39]:
stoplist = set([b for _, b in idf_words[:100]])
sorted(list(stoplist))[:20]

['ale',
 'bardzo',
 'bez',
 'być',
 'będący',
 'cecha',
 'celu',
 'charakterystyczny',
 'ciała',
 'co',
 'coś',
 'czegoś',
 'czymś',
 'często',
 'części',
 'część',
 'człowiek',
 'człowieka',
 'człowieku',
 'dla']

In [26]:
!pip install spacy -U
!python -m spacy download pl_core_news_lg
!python -m spacy link pl_core_news_lg pl_core_news_lg -f

import spacy
nlp = spacy.load('pl_core_news_lg')

     |████████████████████████████████| 10.4MB 4.3MB/s 
     |████████████████████████████████| 1.1MB 47.7MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 604.2MB 1.2MB/s 
  Created wheel for pl-core-news-lg: filename=pl_core_news_lg-2.3.0-cp36-none-any.whl size=604232614 sha256=965e4d10fa3a177efbba3b8e3ce3d798a2c084e1ccf43642ff9a6bdd08895a4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-kt5n511m/wheels/fb/60/e5/4e03ebc777d2bd26ac5394f00db1695bede7a92bc9e7ce1789
Successfully built pl-core-news-lg
✔ Download and installation successful
You can now load the model via spacy.load('pl_core_news_lg')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pl_core_news_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pl_core_news_lg
You can now load th

In [28]:
senses = [item for item in lexical_units.values() if item.name == "ekran"]
senses

[LexicalUnit(id='1599', name='ekran', desc='płaszczyzna, na którą rzuca się obraz, np. w kinie.', pos='rzeczownik'),
 LexicalUnit(id='1600', name='ekran', desc='osłona pełniąca funkcję zabezpieczenia, izolacji przed czymś szkodliwym.', pos='rzeczownik'),
 LexicalUnit(id='380963', name='ekran', desc='rodzaj zasłony (z tkaniny, skóry, żelaza), która bywa ustawiana przed kominkiem.', pos='rzeczownik'),
 LexicalUnit(id='19744', name='ekran', desc='wyświetlacz; powierzchnia, na której jest coś wyświetlane.', pos='rzeczownik'),
 LexicalUnit(id='371009', name='ekran', desc='duża naszywka z nazwą zespołu, która pasuje na środkowy klin jeansowej kurtki lub kamizelki.', pos='rzeczownik')]

In [41]:
def nlp_with_stoplist(text, nlp, stoplist):
  doc = nlp(text)
  words = [t.text for t in doc if t.text.lower() not in stoplist]
  text_without_stoplist = " ".join(words)
  return nlp(text_without_stoplist)

print(text)
doc = nlp_with_stoplist(text.replace("ekranie", "_"), nlp, stoplist)
print(doc)

Jak włączyłem komputer to na ekranie wyświetlił się komunikat o błędzie
włączyłem komputer _ wyświetlił komunikat o błędzie


In [42]:
sims = [(doc.similarity(nlp_with_stoplist(sense.desc.lower(), nlp, stoplist)), sense) for sense in senses]
sims

[(0.290360020906725,
  LexicalUnit(id='1599', name='ekran', desc='płaszczyzna, na którą rzuca się obraz, np. w kinie.', pos='rzeczownik')),
 (0.2560558785663949,
  LexicalUnit(id='1600', name='ekran', desc='osłona pełniąca funkcję zabezpieczenia, izolacji przed czymś szkodliwym.', pos='rzeczownik')),
 (0.197439274616272,
  LexicalUnit(id='380963', name='ekran', desc='rodzaj zasłony (z tkaniny, skóry, żelaza), która bywa ustawiana przed kominkiem.', pos='rzeczownik')),
 (0.3041911323176419,
  LexicalUnit(id='19744', name='ekran', desc='wyświetlacz; powierzchnia, na której jest coś wyświetlane.', pos='rzeczownik')),
 (0.11335399783008535,
  LexicalUnit(id='371009', name='ekran', desc='duża naszywka z nazwą zespołu, która pasuje na środkowy klin jeansowej kurtki lub kamizelki.', pos='rzeczownik'))]

In [46]:
for sim, sense in sorted(sims, reverse=True):
  print("%.3f  %s" % (sim, sense.desc))

0.304  wyświetlacz; powierzchnia, na której jest coś wyświetlane.
0.290  płaszczyzna, na którą rzuca się obraz, np. w kinie.
0.256  osłona pełniąca funkcję zabezpieczenia, izolacji przed czymś szkodliwym.
0.197  rodzaj zasłony (z tkaniny, skóry, żelaza), która bywa ustawiana przed kominkiem.
0.113  duża naszywka z nazwą zespołu, która pasuje na środkowy klin jeansowej kurtki lub kamizelki.
